### Check GPU hardware

In [1]:
!nvidia-smi

Sat Jun  3 07:36:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:9B:00.0 Off |                    0 |
| N/A   28C    P0    67W / 400W |   7411MiB / 81920MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Install requirements for ProtBert embedder

In [2]:
!pip install torch transformers sentencepiece h5py


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
!pip3 install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


### Set up embedding directories 

In [4]:
# Create directory for storing model weights (2.3GB) and example sequences.
# Here we use the encoder-part of ProtT5-XL-U50 in half-precision (fp16) as 
# it performed best in our benchmarks (also outperforming ProtBERT-BFD).
# Also download secondary structure prediction checkpoint to show annotation extraction from embeddings
!mkdir protT5 # root directory for storing checkpoints, results etc
!mkdir protT5/protT5_checkpoint # directory holding the ProtT5 checkpoint
!mkdir protT5/sec_struct_checkpoint # directory storing the supervised classifier's checkpoint
!mkdir protT5/output # directory for storing your embeddings & predictions
!wget -nc -P protT5/ https://rostlab.org/~deepppi/example_seqs.fasta
# Huge kudos to the bio_embeddings team here! We will integrate the new encoder, half-prec ProtT5 checkpoint soon
!wget -nc -P protT5/sec_struct_checkpoint http://data.bioembeddings.com/public/embeddings/feature_models/t5/secstruct_checkpoint.pt

--2023-06-03 07:36:45--  https://rostlab.org/~deepppi/example_seqs.fasta
Resolving rostlab.org (rostlab.org)... 131.159.28.73
Connecting to rostlab.org (rostlab.org)|131.159.28.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 627
Saving to: ‘protT5/example_seqs.fasta’

example_seqs.fasta  100%[===================>]     627  --.-KB/s    in 0s      

2023-06-03 07:36:46 (235 MB/s) - ‘protT5/example_seqs.fasta’ saved [627/627]

--2023-06-03 07:36:46--  http://data.bioembeddings.com/public/embeddings/feature_models/t5/secstruct_checkpoint.pt
Resolving data.bioembeddings.com (data.bioembeddings.com)... 143.95.108.236
Connecting to data.bioembeddings.com (data.bioembeddings.com)|143.95.108.236|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3727951 (3.6M)
Saving to: ‘protT5/sec_struct_checkpoint/secstruct_checkpoint.pt’

secstruct_checkpoin 100%[===================>]   3.55M  2.21MB/s    in 1.6s    

2023-06-03 07:36:48 (2.21 MB/s) - ‘prot

### Download interaction, dictionary/fasta file

In [5]:
# Yous should change for the dataset you want to test
!wget https://raw.githubusercontent.com/anhvt00/MCAPS/master/data/Golden-standard-datasets/Guo-2008/guo_dict.tsv
!wget https://raw.githubusercontent.com/anhvt00/MCAPS/master/data/Golden-standard-datasets/Guo-2008/guo_pairs.tsv
!wget https://raw.githubusercontent.com/anhvt00/MCAPS/master/data/Golden-standard-datasets/Guo-2008/guo.fasta
    
pair_dataset_path = 'guo_pairs.tsv'
seq_path = 'guo.fasta'

--2023-06-03 07:36:54--  https://raw.githubusercontent.com/anhvt00/MCAPS/master/data/Golden-standard-datasets/Guo-2008/guo_dict.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1415079 (1.3M) [text/plain]
Saving to: ‘guo_dict.tsv’

guo_dict.tsv        100%[===================>]   1.35M  6.46MB/s    in 0.2s    

2023-06-03 07:36:55 (6.46 MB/s) - ‘guo_dict.tsv’ saved [1415079/1415079]

--2023-06-03 07:36:55--  https://raw.githubusercontent.com/anhvt00/MCAPS/master/data/Golden-standard-datasets/Guo-2008/guo_pairs.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, a

### Choose sequence FASTA file for embedding 

In [6]:
# In the following you can define your desired output. Current options:
# per_residue embeddings
# per_protein embeddings
# secondary structure predictions

# Replace this file with your own (multi-)FASTA
# Headers are expected to start with ">";
# seq_path = "guo.fasta"

# whether to retrieve embeddings for each residue in a protein 
# --> Lx1024 matrix per protein with L being the protein's length
# as a rule of thumb: 1k proteins require around 1GB RAM/disk
per_residue = True 
per_residue_path = "./protT5/output/per_residue_embeddings.h5" # where to store the embeddings

# whether to retrieve per-protein embeddings 
# --> only one 1024-d vector per protein, irrespective of its length
per_protein = False
per_protein_path = "./protT5/output/per_protein_embeddings.h5" # where to store the embeddings

# whether to retrieve secondary structure predictions
# This can be replaced by your method after being trained on ProtT5 embeddings
sec_struct = False
sec_struct_path = "./protT5/output/ss3_preds.fasta" # file for storing predictions

# make sure that either per-residue or per-protein embeddings are stored
assert per_protein is True or per_residue is True or sec_struct is True, print(
    "Minimally, you need to active per_residue, per_protein or sec_struct. (or any combination)")


### Read FASTA file function

In [7]:
def read_fasta( fasta_path, split_char="!", id_field=0):
    '''
        Reads in fasta file containing multiple sequences.
        Split_char and id_field allow to control identifier extraction from header.
        E.g.: set split_char="|" and id_field=1 for SwissProt/UniProt Headers.
        Returns dictionary holding multiple sequences or only single 
        sequence, depending on input file.
    '''
    
    seqs = dict()
    with open( fasta_path, 'r' ) as fasta_f:
        for line in fasta_f:
            # get uniprot ID from header and create new entry
            if line.startswith('>'):
                uniprot_id = line.replace('>', '').strip().split(split_char)[id_field]
                # replace tokens that are mis-interpreted when loading h5
                uniprot_id = uniprot_id.replace("/","_").replace(".","_")
                seqs[ uniprot_id ] = ''
            else:
                # repl. all whie-space chars and join seqs spanning multiple lines, drop gaps and cast to upper-case
                seq= ''.join( line.split() ).upper().replace("-","")
                # repl. all non-standard AAs and map them to unknown/X
                seq = seq.replace('U','X').replace('Z','X').replace('O','X')
                seqs[ uniprot_id ] += seq 
    example_id=next(iter(seqs))
    print("Read {} sequences.".format(len(seqs)))
    print("Example:\n{}\n{}".format(example_id,seqs[example_id]))

    return seqs

### Import ProtBert embedder model 

In [8]:
from transformers import T5EncoderModel, T5Tokenizer
import torch
import numpy as np
import h5py
import time
from tqdm import tqdm
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("Using {}".format(device))

def get_T5_model():
    model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_half_uniref50-enc")
    model = model.to(device) # move model to GPU
    model = model.eval() # set model to evaluation model
    tokenizer = T5Tokenizer.from_pretrained('Rostlab/prot_t5_xl_half_uniref50-enc', do_lower_case=False)

    return model, tokenizer

Using cuda:0


### Generate embeddings function

In [9]:
# Generate embeddings via batch-processing
# per_residue indicates that embeddings for each residue in a protein should be returned.
# per_protein indicates that embeddings for a whole protein should be returned (average-pooling)
# max_residues gives the upper limit of residues within one batch
# max_seq_len gives the upper sequences length for applying batch-processing
# max_batch gives the upper number of sequences per batch
def get_embeddings( model, tokenizer, seqs, per_residue, per_protein, sec_struct, 
                   max_residues=4000, max_seq_len=1000, max_batch=100 ):

    if sec_struct:
      sec_struct_model = load_sec_struct_model()

    results = {"residue_embs" : dict(), 
               "protein_embs" : dict(),
               "sec_structs" : dict() 
               }

    # sort sequences according to length (reduces unnecessary padding --> speeds up embedding)
    seq_dict   = sorted( seqs.items(), key=lambda kv: len( seqs[kv[0]] ), reverse=True )
    start = time.time()
    batch = list()
    for seq_idx, (pdb_id, seq) in tqdm(enumerate(seq_dict,1)):
        seq = seq
        seq_len = len(seq)
        seq = ' '.join(list(seq))
        batch.append((pdb_id,seq,seq_len))

        # count residues in current batch and add the last sequence length to
        # avoid that batches with (n_res_batch > max_residues) get processed 
        n_res_batch = sum([ s_len for  _, _, s_len in batch ]) + seq_len 
        if len(batch) >= max_batch or n_res_batch>=max_residues or seq_idx==len(seq_dict) or seq_len>max_seq_len:
            pdb_ids, seqs, seq_lens = zip(*batch)
            batch = list()

            # add_special_tokens adds extra token at the end of each sequence
            token_encoding = tokenizer.batch_encode_plus(seqs, add_special_tokens=True, padding="longest")
            input_ids      = torch.tensor(token_encoding['input_ids']).to(device)
            attention_mask = torch.tensor(token_encoding['attention_mask']).to(device)
            
            try:
                with torch.no_grad():
                    # returns: ( batch-size x max_seq_len_in_minibatch x embedding_dim )
                    embedding_repr = model(input_ids, attention_mask=attention_mask)
            except RuntimeError:
                print("RuntimeError during embedding for {} (L={})".format(pdb_id, seq_len))
                continue

            if sec_struct: # in case you want to predict secondary structure from embeddings
              d3_Yhat, d8_Yhat, diso_Yhat = sec_struct_model(embedding_repr.last_hidden_state)


            for batch_idx, identifier in enumerate(pdb_ids): # for each protein in the current mini-batch
                s_len = seq_lens[batch_idx]
                # slice off padding --> batch-size x seq_len x embedding_dim  
                emb = embedding_repr.last_hidden_state[batch_idx,:s_len]
                if sec_struct: # get classification results
                    results["sec_structs"][identifier] = torch.max( d3_Yhat[batch_idx,:s_len], dim=1 )[1].detach().cpu().numpy().squeeze()
                if per_residue: # store per-residue embeddings (Lx1024)
                    results["residue_embs"][ identifier ] = emb.detach().cpu().numpy().squeeze()
                if per_protein: # apply average-pooling to derive per-protein embeddings (1024-d)
                    protein_emb = emb.mean(dim=0)
                    results["protein_embs"][identifier] = protein_emb.detach().cpu().numpy().squeeze()


    passed_time=time.time()-start
    avg_time = passed_time/len(results["residue_embs"]) if per_residue else passed_time/len(results["protein_embs"])
    print('\n############# EMBEDDING STATS #############')
    print('Total number of per-residue embeddings: {}'.format(len(results["residue_embs"])))
    print('Total number of per-protein embeddings: {}'.format(len(results["protein_embs"])))
    print("Time for generating embeddings: {:.1f}[m] ({:.3f}[s/protein])".format(
        passed_time/60, avg_time ))
    print('\n############# END #############')
    return results

### Write embeddings to disk function

In [10]:
def save_embeddings(emb_dict,out_path):
    with h5py.File(str(out_path), "w") as hf:
        for sequence_id, embedding in emb_dict.items():
            # noinspection PyUnboundLocalVariable
            hf.create_dataset(sequence_id, data=embedding)
    return None

### Generate embedding for FASTA file

In [11]:
# Load the encoder part of ProtT5-XL-U50 in half-precision (recommended)
model, tokenizer = get_T5_model()

# Load example fasta.
seqs = read_fasta( seq_path )

for id, seq in seqs.items():
    if len(seq) > 1200:
        seqs[id] = seq[:1200]


# Compute embeddings and/or secondary structure predictions
results = get_embeddings( model, tokenizer, seqs,
                         per_residue, per_protein, sec_struct)

# Store per-residue embeddings
if per_residue:
  save_embeddings(results["residue_embs"], per_residue_path)
if per_protein:
  save_embeddings(results["protein_embs"], per_protein_path)

Read 2497 sequences.
Example:
O13297
MSYTDNPPQTKRALSLDDLVNHDENEKVKLQKLSEAANGSRPFAENLESDINQTETGQAAPIDNYKESTGHGSHSQKPKSRKSSNDDEETDTDDEMGASGEINFDSEMDFDYDKQHRNLLSNGSPPMNDGSDANAKLEKPSDDSIHQNSKSDEEQRIPKQGNEGNIASNYITQVPLQKQKQTEKKIAGNAVGSVVKKEEEANAAVDNIFEEKATLQSKKNNIKRDLEVLNEISASSKPSKYKNVPIWAQKWKPTIKALQSINVKDLKIDPSFLNIIPDDDLTKSVQDWVYATIYSIAPELRSFIELEMKFGVIIDAKGPDRVNPPVSSQCVFTELDAHLTPNIDASLFKELSKYIRGISEVTENTGKFSIIESQTRDSVYRVGLSTQRPRFLRMSTDIKTGRVGQFIEKRHVAQLLLYSPKDSYDVKISLNLELPVPDNDPPEKYKSQSPISERTKDRVSYIHNDSCTRIDITKVENHNQNSKSRQSETTHEVELEINTPALLNAFDNITNDSKEYASLIRTFLNNGTIIRRKLSSLSYEIFEGSKKVM


2497it [01:27, 28.54it/s] 



############# EMBEDDING STATS #############
Total number of per-residue embeddings: 2497
Total number of per-protein embeddings: 0
Time for generating embeddings: 1.5[m] (0.035[s/protein])

############# END #############


### Import libraries

In [12]:
!pip install xgboost
!pip install matplotlib
!apt install wget
!pip install scikit-learn pandas 
!pip install gdown
import numpy as np
from xgboost import XGBClassifier
import xgboost as xgb
import math
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Input, Dropout, BatchNormalization, ReLU, LeakyReLU, Conv1D, GlobalMaxPooling1D, AveragePooling1D, MaxPooling1D, GlobalAveragePooling1D
from tensorflow.keras.layers import concatenate, multiply, Bidirectional, LSTM, GRU, Flatten, PReLU, add, SpatialDropout1D
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2, l1_l2
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, precision_recall_curve
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef,accuracy_score, precision_score,recall_score
from sklearn.manifold import TSNE
import os
from keras.callbacks import ModelCheckpoint
from tensorflow.keras import mixed_precision
!pip install tensorflow-addons
import tensorflow_addons as tfa

import time

from tqdm import tqdm

### Setting RAM GPU for training growth 
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

# Disables caching (when set to 1) or enables caching (when set to 0) for just-in-time-compilation. When disabled,
# no binary code is added to or retrieved from the cache.
os.environ['CUDA_CACHE_DISABLE'] = '0' # orig is 0

# When set to 1, forces the device driver to ignore any binary code embedded in an application 
# (see Application Compatibility) and to just-in-time compile embedded PTX code instead.
# If a kernel does not have embedded PTX code, it will fail to load. This environment variable can be used to
# validate that PTX code is embedded in an application and that its just-in-time compilation works as expected to guarantee application 
# forward compatibility with future architectures.
os.environ['CUDA_FORCE_PTX_JIT'] = '1'# no orig


os.environ['HOROVOD_GPU_ALLREDUCE'] = 'NCCL'

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

os.environ['TF_GPU_THREAD_MODE'] = 'gpu_private'
os.environ['TF_GPU_THREAD_COUNT']='1'

os.environ['TF_USE_CUDNN_BATCHNORM_SPATIAL_PERSISTENT'] = '1'

os.environ['TF_ADJUST_HUE_FUSED'] = '1'
os.environ['TF_ADJUST_SATURATION_FUSED'] = '1'
os.environ['TF_ENABLE_WINOGRAD_NONFUSED'] = '1'

os.environ['TF_SYNC_ON_FINISH'] = '0'
os.environ['TF_AUTOTUNE_THRESHOLD'] = '2'
os.environ['TF_DISABLE_NVTX_RANGES'] = '1'
os.environ["TF_ENABLE_AUTO_MIXED_PRECISION_GRAPH_REWRITE"] = "1"



# =================================================
mixed_precision.set_global_policy('mixed_float16')


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip
Reading package lists... Done
Building dependency tree       
Reading state information... Done
wget is already the newest version (1.20.3-1ubuntu2).
0 upgraded, 0 newly installed, 0 to remove and 46 not upgraded.

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


2023-06-03 07:39:44.150366: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


/usr/local/lib/python3.8/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
2023-06-03 07:39:49.319426: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 70636 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:9b:00.0, compute capability: 8.0


1 Physical GPUs, 1 Logical GPUs
INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA A100-SXM4-80GB, compute capability 8.0


### Read interaction dataset to DataFrame

In [2]:
pair_dataframe = pd.read_csv(pair_dataset_path, sep='\t', header=None)
pair_array  = pair_dataframe.to_numpy()
np.random.seed(42)
np.random.shuffle(pair_array)
pair_dataframe = pd.DataFrame(pair_array)
pair_dataframe = pd.DataFrame(pair_array, columns = ['p1', 'p2', 'label'])
pair_dataframe['label'] = pair_dataframe['label'].astype('float16') 
pair_dataframe

,p1,p2,label
0,Q12236,P36094,1.0
1,P39730,P00899,1.0
2,P32389,P09734,0.0
3,P35817,P25502,1.0
4,P40956,P15705,1.0
...,...,...,...
11183,Q12477,Q07540,0.0
11184,P20459,P19454,1.0
11185,P40465,P30771,1.0
11186,P36224,P30619,1.0


### Padding function 

In [3]:
def pad(rst, length=1200, dim=1024):
    if len(rst) > length:
        return rst[:length]
    elif len(rst) < length:
        return np.concatenate((rst, np.zeros((length - len(rst), dim))))
    return rst

### Read embedding matrix

In [6]:
import h5py
embedding_matrix= h5py.File(per_residue_path, 'r')
protein_keys = list(embedding_matrix.keys())
embedding_dict = dict()

for key in protein_keys:
  embedding_dict[key] = np.array(embedding_matrix[key])

### Split indicies for cross-validation

In [7]:
from sklearn.model_selection import KFold, ShuffleSplit
from sklearn.model_selection import train_test_split
kf = KFold(n_splits=5, random_state=42, shuffle=True)
train_test = []
for train, test in kf.split(pair_dataframe['label']):
  train_test.append((train, test))

### Architecture of MCAPS

In [8]:
from tensorflow.keras.utils import get_custom_objects
def leaky_relu(x, alpha = .2):
   return tf.keras.backend.maximum(alpha*x, x)
!pip install tensorflow-addons
import tensorflow_addons as tfa 
get_custom_objects().update({'leaky_relu': leaky_relu})
get_custom_objects().update({'mish': tfa.activations.mish})
get_custom_objects().update({'lisht': tfa.activations.lisht})
get_custom_objects().update({'rrelu': tfa.activations.rrelu})


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [14]:
seq_size = 1200
dim = 1024
def multi_cnn():
    DEPTH = 5
    WIDTH = 3
    POOLING_SIZE = 4
    FILTERS = 50
    KERNEL_SIZE = 2
    DEPTH_DENSE1 = 3
    DEPTH_DENSE2 = 2
    DROPOUT = DROPOUT1 = DROPOUT2 = 0.05
    DROPOUT_SPATIAL= 0.15
    ACTIVATION = 'swish'
    ACTIVATION_CNN = 'swish'
    INITIALIZER = 'glorot_normal'
    
    def BlockCNN_single(KERNEL_SIZE, POOLING_SIZE, FILTERS, LAYER_IN1, LAYER_IN2):
        c1 = Conv1D(filters=FILTERS, kernel_size=KERNEL_SIZE, activation=ACTIVATION_CNN, padding='same')
        x1 = c1(LAYER_IN1)
        x2 = c1(LAYER_IN2)

        g1 = Dropout(DROPOUT)(concatenate([GlobalMaxPooling1D()(x1),GlobalAveragePooling1D()(x1)]))
        a1 = GlobalAveragePooling1D()(x1)
        g2 = Dropout(DROPOUT)(concatenate([GlobalMaxPooling1D()(x2),GlobalAveragePooling1D()(x2)]))
        a2 = GlobalAveragePooling1D()(x1)

        x1 = SpatialDropout1D(DROPOUT_SPATIAL)(concatenate([MaxPooling1D(POOLING_SIZE)(x1), AveragePooling1D(POOLING_SIZE)(x1)]))
        x2 = SpatialDropout1D(DROPOUT_SPATIAL)(concatenate([MaxPooling1D(POOLING_SIZE)(x2), AveragePooling1D(POOLING_SIZE)(x2)]))

        return x1, x2, g1, g2, a1, a2

    def BlockCNN_multi(POOLING_SIZE, FILTERS, LAYER_IN1, LAYER_IN2, WIDTH):
      X1 = []
      X2 = []
      G1 = []
      G2 = []
      A1 = []
      A2 = []
      for i in range(2, 2+WIDTH):
        x1, x2, g1, g2, a1, a2 = BlockCNN_single(i, POOLING_SIZE, FILTERS, LAYER_IN1, LAYER_IN2)
        X1.append(x1)
        X2.append(x2)
        G1.append(g1)
        G2.append(g2)
        A1.append(a1)
        A2.append(a2)
      x1 = concatenate(X1)
      x2 = concatenate(X2)
      g1 = GlobalMaxPooling1D()(x1)
      g2 = GlobalMaxPooling1D()(x2)
      return x1, x2, g1, g2

    def BlockCNN_single_deep(KERNEL_SIZE, POOLING_SIZE, DEPTH, FILTERS, LAYER_IN1, LAYER_IN2):
      X1 = []
      X2 = []
      G1 = []
      G2 = []
      A1 = []
      A2 = []
      x1 = LAYER_IN1
      x2 = LAYER_IN2
      for i in range(DEPTH):
        x1, x2, g1, g2, a1, a2 = BlockCNN_single(KERNEL_SIZE, POOLING_SIZE, FILTERS, x1, x2)
        X1.append(x1)
        X2.append(x2)
        G1.append(g1)
        G2.append(g2)
        A1.append(a1)
        A2.append(a2)

      return X1, X2, G1, G2, A1, A2

    input1 = Input(shape=(seq_size, dim), name="seq1")
    input2 = Input(shape=(seq_size, dim), name="seq2")
    


    X1 = dict()
    X2 = dict()
    G1 = dict()
    G2 = dict()
    A1 = dict()
    A2 = dict()

    for i in range(KERNEL_SIZE, KERNEL_SIZE+WIDTH):
      X1[f'{i}'], X2[f'{i}'], G1[f'{i}'], G2[f'{i}'], A1[f'{i}'], A2[f'{i}'] = BlockCNN_single_deep(i, POOLING_SIZE, DEPTH, FILTERS, input1, input2)

    s1 = []
    s2 = []
    for i in range(KERNEL_SIZE, KERNEL_SIZE+WIDTH):
      s1.extend(G1[f'{i}'])
      s2.extend(G2[f'{i}'])

    s1 = concatenate(s1)
    s2 = concatenate(s2)
    
    s1 = BatchNormalization(momentum=.9)(s1)
    s2 = BatchNormalization(momentum=.9)(s2)

    s1 = Dropout(DROPOUT1)(s1)
    s2 = Dropout(DROPOUT1)(s2)
    
    s1_shape = s1.shape[-1]
    DENSE1 = 744 
    d1 = []
    for i in range(DEPTH_DENSE1):
        d1.append(Dense(int(DENSE1*(1/2)**i), kernel_initializer=INITIALIZER, activation=ACTIVATION))

    for i in range(DEPTH_DENSE1):
        s1 = d1[i](s1)
        s2 = d1[i](s2)
        s1 = Dropout(DROPOUT1)(s1)
        s2 = Dropout(DROPOUT1)(s2)
        
    s = concatenate([s1, s2])

    
    s_shape = s.shape[-1]
    DENSE2 = 328
        
    d2 = []
    for i in range(DEPTH_DENSE2):
        d2.append(Dense(int(DENSE2*(1/2)**i), kernel_initializer=INITIALIZER, activation=ACTIVATION))

    for i in range(DEPTH_DENSE2):
        s = d2[i](s)
        s = Dropout(DROPOUT2)(s)

    output = Dense(1, activation='sigmoid')(s)
    model = Model(inputs=[input1, input2], outputs=[output])
    
    adam = Adam(learning_rate=1e-3, amsgrad=True, epsilon=1e-6)
    model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
    return model


model = multi_cnn()
model.summary()
tf.keras.utils.plot_model(model, show_shapes=True)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 seq1 (InputLayer)              [(None, 1200, 1024)  0           []                               
                                ]                                                                 
                                                                                                  
 seq2 (InputLayer)              [(None, 1200, 1024)  0           []                               
                                ]                                                                 
                                                                                                  
 conv1d_45 (Conv1D)             (None, 1200, 50)     102450      ['seq1[0][0]',                   
                                                                  'seq2[0][0]']             

### Function for mapping in dataset pipeline

In [10]:
def func(i):
    i = i.numpy() # Decoding from the EagerTensor object
    x1= pad(embedding_dict[pair_dataframe['p1'][i]])
    x2= pad(embedding_dict[pair_dataframe['p2'][i]])
    y = pair_dataframe['label'][i]
    return x1, x2, y

def _fixup_shape(x1, x2, y):
    x1.set_shape((seq_size, dim))
    x2.set_shape((seq_size, dim)) 
    y.set_shape(()) 

    return (x1, x2), y

def pad(rst, length=1200, dim=1024):
    if len(rst) > length:
        return rst[:length]
    elif len(rst) < length:
        return np.concatenate((rst, np.zeros((length - len(rst), dim))))
    return rst

### 5-fold cross validation DL

In [11]:
# Use intermediate layer to transform pairs matrix
BATCH_SIZE = 64
pred_dataset = tf.data.Dataset.from_generator(lambda: range(len(pair_dataframe)), tf.uint32)

pred_dataset = pred_dataset.map(lambda i: tf.py_function(func=func, 
                                              inp=[i], 
                                              Tout=[tf.float16,
                                                    tf.float16, tf.float16]
                                              ), 
                      num_parallel_calls=tf.data.AUTOTUNE).map(_fixup_shape).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [15]:
##### Hyperparameter
EPOCHS=60
DTYPE = 'float16'



acc_list = []
recall_list = []
prec_list = []
spec_list = []
f1_list = []
mcc_list = []
auc_list = []
prc_list = []
dnn_fpr_list = []
dnn_tpr_list = []
dnn_fpr = []  
dnn_tpr = []
dnn_prc_prec = []
dnn_prc_recall = []
FOLD = 1
for train, test in train_test:
  print(f"=========================== FOLD {FOLD} ==================================")
  train_dataset = tf.data.Dataset.from_generator(lambda: train, tf.uint32)

  train_dataset = train_dataset.map(lambda i: tf.py_function(func=func, 
                                                inp=[i], 
                                                Tout=[DTYPE,DTYPE, DTYPE
                                                      ]
                                                ), 
                        num_parallel_calls=tf.data.AUTOTUNE).map(_fixup_shape).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

  val_dataset = tf.data.Dataset.from_generator(lambda: test, tf.uint32)

  val_dataset = val_dataset.map(lambda i: tf.py_function(func=func, 
                                                inp=[i], 
                                                Tout=[DTYPE,
                                                      DTYPE, DTYPE]
                                                ), 
                        num_parallel_calls=tf.data.AUTOTUNE).map(_fixup_shape).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
  checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=f'my_best_model_fold_{FOLD}.hdf5', monitor='val_loss',verbose=1, save_best_only=True, mode='min')
  model = multi_cnn()

  model.fit(train_dataset, epochs=EPOCHS, validation_data=val_dataset, callbacks=[checkpoint])

  test_dataset = tf.data.Dataset.from_generator(lambda: test, tf.uint32)
  test_dataset = test_dataset.map(lambda i: tf.py_function(func=func, 
                                              inp=[i], 
                                              Tout=[DTYPE,
                                                    DTYPE, DTYPE]
                                              ), 
                      num_parallel_calls=tf.data.AUTOTUNE).map(_fixup_shape).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

  pred = model.predict(test_dataset)
  cm1=confusion_matrix(pair_dataframe['label'][test].values, np.round(pred))
  acc = (cm1[0,0]+cm1[1,1])/(cm1[0,0]+cm1[0,1]+cm1[1,0]+cm1[1,1])
  spec= (cm1[0,0])/(cm1[0,0]+cm1[0,1])
  sens = (cm1[1,1])/(cm1[1,0]+cm1[1,1])
  prec=cm1[1,1]/(cm1[1,1]+cm1[0,1])
  rec=cm1[1,1]/(cm1[1,1]+cm1[1,0])
  f1 = 2 * (prec * rec) / (prec + rec)
  mcc = matthews_corrcoef(pair_dataframe['label'][test].values, np.round(pred))

  fpr, tpr, _= roc_curve(pair_dataframe['label'][test].values[:], pred[:])   
  prc_prec, prc_recall, thresholds = precision_recall_curve(pair_dataframe['label'][test].values[:], pred[:])   
  auc = metrics.roc_auc_score(pair_dataframe['label'][test].values, pred)
  prc = metrics.average_precision_score(pair_dataframe['label'][test].values, pred)
  acc_list.append(acc)
  recall_list.append(rec)
  prec_list.append(prec)
  spec_list.append(spec)
  f1_list.append(f1)
  mcc_list.append(mcc)

  dnn_fpr_list.append(fpr)
  dnn_tpr_list.append(tpr)
  auc_list.append(auc)
  prc_list.append(prc)
  FOLD += 1
  print("======================= manual compute ==============")
  print (f'accuracy: {acc}, precision: {prec}, recall: {rec}, specificity: {spec}, mcc: {mcc} ,f1-score: {f1}, auc: {auc}, prc: {prc}')





=========================== FOLD 1 ==================================
Epoch 1/60


2023-06-02 02:40:19.067480: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-02 02:40:19.067715: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-02 02:40:30.556788: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x1777ad7a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-06-02 02:40:30.556859: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
202

    140/Unknown - 51s 248ms/step - loss: 0.5720 - accuracy: 0.6914

2023-06-02 02:41:10.399904: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-02 02:41:10.400127: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]



Epoch 1: val_loss improved from inf to 0.45518, saving model to my_best_model_fold_1.hdf5
140/140 [==============================] - 61s 318ms/step - loss: 0.5720 - accuracy: 0.6914 - val_loss: 0.4552 - val_accuracy: 0.8088
Epoch 2/60
140/140 [==============================] - ETA: 0s - loss: 0.3209 - accuracy: 0.8682
Epoch 2: val_loss improved from 0.45518 to 0.23755, saving model to my_best_model_fold_1.hdf5
140/140 [==============================] - 44s 317ms/step - loss: 0.3209 - accuracy: 0.8682 - val_loss: 0.2375 - val_accuracy: 0.9138
Epoch 3/60
140/140 [==============================] - ETA: 0s - loss: 0.2176 - accuracy: 0.9102
Epoch 3: val_loss improved from 0.23755 to 0.16428, saving model to my_best_model_fold_1.hdf5
140/140 [==============================] - 44s 314ms/step - loss: 0.2176 - accuracy: 0.9102 - val_loss: 0.1643 - val_accuracy: 0.9321
Epoch 4/60
140/140 [==============================] - ETA: 0s - loss: 0.1844 - accuracy: 0.9215
Epoch 4: val_loss improved from

...

    140/Unknown - 50s 244ms/step - loss: 0.5857 - accuracy: 0.6869

2023-06-02 05:37:14.921779: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-02 05:37:14.922014: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]



Epoch 1: val_loss improved from inf to 0.44978, saving model to my_best_model_fold_5.hdf5
140/140 [==============================] - 59s 312ms/step - loss: 0.5857 - accuracy: 0.6869 - val_loss: 0.4498 - val_accuracy: 0.8207
Epoch 2/60
140/140 [==============================] - ETA: 0s - loss: 0.3395 - accuracy: 0.8599
Epoch 2: val_loss improved from 0.44978 to 0.25489, saving model to my_best_model_fold_5.hdf5
140/140 [==============================] - 43s 309ms/step - loss: 0.3395 - accuracy: 0.8599 - val_loss: 0.2549 - val_accuracy: 0.9017
Epoch 3/60
140/140 [==============================] - ETA: 0s - loss: 0.2264 - accuracy: 0.9082
Epoch 3: val_loss improved from 0.25489 to 0.19102, saving model to my_best_model_fold_5.hdf5
140/140 [==============================] - 43s 310ms/step - loss: 0.2264 - accuracy: 0.9082 - val_loss: 0.1910 - val_accuracy: 0.9133
Epoch 4/60
140/140 [==============================] - ETA: 0s - loss: 0.1871 - accuracy: 0.9200
Epoch 4: val_loss improved from

2023-06-02 06:19:56.154855: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-02 06:19:56.155089: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


35/35 [==============================] - 9s 240ms/step
======================= manual compute ==============
accuracy: 0.9865891819400984, precision: 0.9871677360219981, recall: 0.9853613906678865, specificity: 0.9877622377622378, mcc: 0.973164878559456 ,f1-score: 0.9862637362637363, auc: 0.9932481173903863, prc: 0.9901856718838212


In [17]:

mean_acc = np.mean(acc_list)
std_acc = np.std(acc_list)
# var_acc = np.var(acc_list)
print("Accuracy: " + str(mean_acc) + " ± " + str(std_acc))
# print("Accuracy_Var:"+str(mean_acc)+" ± "+str(var_acc))

mean_spec = np.mean(spec_list)
std_spec = np.std(spec_list)
print("Specificity: " + str(mean_spec) + " ± " + str(std_spec))

mean_prec = np.mean(prec_list)
std_prec = np.std(prec_list)
print("Precision: " + str(mean_prec) + " ± " + str(std_prec))

mean_recall = np.mean(recall_list)
std_recall = np.std(recall_list)
print("Recall: " + str(mean_recall) + " ± " + str(std_recall))

mean_mcc = np.mean(mcc_list)
std_mcc = np.std(mcc_list)
print("MCC: " + str(mean_mcc) + " ± " + str(std_mcc))

mean_auc = np.mean(auc_list)
std_auc = np.std(auc_list)
print("AUC: " + str(mean_auc)  + " ± " + str(std_auc))

mean_prc = np.mean(prc_list)
std_prc = np.std(prc_list)
print("PRC:" + str(mean_prc)  + " ± " + str(std_prc)) 

mean_f1 = np.mean(f1_list)
std_f1 = np.std(f1_list)
print("F1_Score: " + str(mean_f1)  + " ± " + str(std_f1)) 

print(str(mean_acc) + "\t" + str(mean_prec) + "\t" + str(mean_recall) + "\t" + str(mean_spec) + "\t" + str(mean_f1) + "\t" + str(mean_mcc) + "\t" + str(mean_auc) + "\t" + str(mean_prc))

Accuracy: 0.9858777733967241 ± 0.002719185119429832
Specificity: 0.9846214314803199 ± 0.003053143629873357
Precision: 0.9846537476939299 ± 0.0031221751648232887
Recall: 0.9870574564388639 ± 0.0038838611160628487
MCC: 0.9717464817861832 ± 0.005454518623884845
AUC: 0.993853177184341 ± 0.00040063625049161834
PRC:0.9908459036770327 ± 0.0009313701725177212
F1_Score: 0.9858505375655113 ± 0.0029741656678989557
0.9858777733967241	0.9846537476939299	0.9870574564388639	0.9846214314803199	0.9858505375655113	0.9717464817861832	0.993853177184341	0.9908459036770327


### 5-fold cross validation Hybrid model

In [18]:
# Use intermediate layer to transform pairs matrix
pred_dataset = tf.data.Dataset.from_generator(lambda: range(len(pair_dataframe)), tf.uint32)

pred_dataset = pred_dataset.map(lambda i: tf.py_function(func=func, 
                                              inp=[i], 
                                              Tout=[tf.float16,
                                                    tf.float16, tf.float16]
                                              ), 
                      num_parallel_calls=tf.data.AUTOTUNE).map(_fixup_shape).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [20]:
intermediate_layer_model = Model(inputs=model.input,outputs=model.get_layer(model.layers[-2].name).output)

# Use intermediate layer to transform pairs matrix
pred = intermediate_layer_model.predict(pred_dataset)
p_merge=pd.DataFrame(pred)    
Trainlabels = pair_dataframe['label']
# create dataframe use transformed pairs matrix outputs and labels
X_train_feat=pd.concat((p_merge,pd.DataFrame(pd.DataFrame(Trainlabels))),axis=1,ignore_index=True)

# write to file dataframe of transformed pairs matrix and labels
X_train_feat.to_csv('X_train.csv',header=False, index=False)

# read dataframe of transformed pairs matrix and labels
Train=pd.read_csv("X_train.csv",header=None)
# Train=Train.sample(frac=1)

X=Train.iloc[:,0:164].values
y=Train.iloc[:,164:].values

extracted_df=X_train_feat


y = y.reshape(-1, )

accuracy = []
specificity = []
sensitivity = []
precision=[]
recall=[]
m_coef=[]

auc_list=[]
prc_list=[]
training_time = 1
kf=StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
for train_, test_ in kf.split(X,y):
    model_= XGBClassifier(booster='gbtree', reg_lambda=1, alpha=1e-7, subsample=0.8, colsample_bytree=0.2, n_estimators=1000, max_depth=5, min_child_weight=2, gamma=1e-7, eta=1e-6)
    print(f"========================== START TRAINING TIME {training_time} ===========================")
    hist=model_.fit(X[train_], y[train_],eval_set=[(X[test_], y[test_])],verbose=False)
    y_pred = model_.predict_proba(X[test_])
    y_true = tf.keras.utils.to_categorical(y[test_]) 
    cm1=confusion_matrix(y_true.argmax(axis=1), y_pred.argmax(axis=1))
    acc = (cm1[0,0]+cm1[1,1])/(cm1[0,0]+cm1[0,1]+cm1[1,0]+cm1[1,1])
    spec= (cm1[0,0])/(cm1[0,0]+cm1[0,1])
    sens = (cm1[1,1])/(cm1[1,0]+cm1[1,1])
    prec=cm1[1,1]/(cm1[1,1]+cm1[0,1])
    rec=cm1[1,1]/(cm1[1,1]+cm1[1,0])
    f1 = 2 * (prec * rec) / (prec + rec)
    coef=matthews_corrcoef(y_true.argmax(axis=1), y_pred.argmax(axis=1), sample_weight=None)
    auc = metrics.roc_auc_score(y_true, y_pred)
    prc = metrics.average_precision_score(y_true, y_pred) # Added

    auc_list.append(auc)
    prc_list.append(prc)
    m_coef.append(coef)
    sensitivity.append(sens)
    specificity.append(spec)
    accuracy.append(acc)
    precision.append(prec)
    recall.append(rec)
    f1_list.append(f1)
    print(f"========================== PERFOMANCE METRICS AT TRAINING TIME {training_time} ===========================")
    print(f"accuracy: {acc}, specificity: {spec}, sensitivity: {sens}, precision= {prec}, recall: {rec} ")
#     if max_accuracy<acc:
#         max_accuracy=acc
#         xgb_fpr=fpr
#         xgb_tpr=tpr
    training_time += 1
        
print("====================================== END CROSS VALIDATION ===============================\n")


mean_acc=np.mean(accuracy)
std_acc=np.std(accuracy)
var_acc=np.var(accuracy)
print("Accuracy:"+str(mean_acc)+" Â± "+str(std_acc))
print("Accuracy_Var:"+str(mean_acc)+" Â± "+str(var_acc))
mean_spec=np.mean(specificity)
std_spec=np.std(specificity)
print("Specificity:"+str(mean_spec)+" Â± "+str(std_spec))
mean_sens=np.mean(sensitivity)
std_sens=np.std(sensitivity)
print("Sensitivity:"+str(mean_sens)+" Â± "+str(std_sens))
mean_prec=np.mean(precision)
std_prec=np.std(precision)
print("Precison:"+str(mean_prec)+" Â± "+str(std_prec))
mean_rec=np.mean(recall)
std_rec=np.std(recall)
print("Recall:"+str(mean_rec)+" Â± "+str(std_rec))
mean_coef=np.mean(m_coef)
std_coef=np.std(m_coef)
print("MCC:"+str(mean_coef)+" Â± "+str(std_coef))

mean_auc = np.mean(auc_list)
std_auc = np.std(auc_list)
print("AUC: " + str(mean_auc) + " Â± " + str(std_auc))

mean_prc = np.mean(prc_list)
std_prc = np.std(prc_list)
print("PRC: " + str(mean_prc) + " Â± " + str(std_prc))

mean_f1 = np.mean(f1_list)
std_f1 = np.std(f1_list)
print("F1_Score: " + str(mean_f1)  + " ± " + str(std_f1)) 

end2 = time.time()
# print(f"Runtime of the program is {end2 - start2}")
# print(f"Total runtime of both DL and ML is {end2 - start2 + mean_train_time}")

print(str(mean_acc) + "\t" + str(mean_prec) + "\t" + str(mean_rec) + "\t" + str(mean_spec) + "\t" + str(mean_f1) \
      + "\t" + str(mean_coef) + "\t" + str(mean_auc) + "\t" + str(mean_prc))


175/175 [==============================] - 41s 229ms/step
========================== START TRAINING TIME 1 ===========================
========================== PERFOMANCE METRICS AT TRAINING TIME 1 ===========================
accuracy: 0.9973190348525469, specificity: 0.9973190348525469, sensitivity: 0.9973190348525469, precision= 0.9973190348525469, recall: 0.9973190348525469 
========================== START TRAINING TIME 2 ===========================
========================== PERFOMANCE METRICS AT TRAINING TIME 2 ===========================
accuracy: 0.9964253798033958, specificity: 0.998212689901698, sensitivity: 0.9946380697050938, precision= 0.9982062780269059, recall: 0.9946380697050938 
========================== START TRAINING TIME 3 ===========================
========================== PERFOMANCE METRICS AT TRAINING TIME 3 ===========================
accuracy: 0.9955317247542449, specificity: 0.9964253798033958, sensitivity: 0.9946380697050938, precision= 0.99641897940913